## Install transformer

In [1]:
! pip install transformers==4.28.0 datasets accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

## Import libraries

In [2]:
from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import torch

import transformers
from transformers import AutoTokenizer, TrainingArguments, Trainer, default_data_collator
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering

import collections
from tqdm.auto import tqdm


## load dataset

In [3]:
train_data, validate_data = load_dataset("squad_v1_pt", split='train[:12%]'), load_dataset('squad_v1_pt', split='validation[:50%]')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad_v1_pt downloaded and prepared to /root/.cache/huggingface/datasets/squad_v1_pt/default/1.1.0/65162e0fbe44f19a4d2ad9f5f507d2e965e74249fc3239dc78b4e3bd93bab7c4. Subsequent calls will reuse this data.


In [4]:
train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10512
})

In [5]:
validate_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5285
})

In [6]:
train_data[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda &quot;Venite Ad Me Omnes&quot;. Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.',
 'question': 'A quem a Virgem Maria supostamente apareceu em 1858 em Lourdes, na França?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [0]}}

## fine-tune the task

In [7]:
sciq_2 = False
model_checkpoint = "roberta-base"
batch_size = 16

## just to see how dataset look like

In [8]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(train_data)

,id,title,context,question,answers
0,56ce5125aab44d1400b886ae,Sino-Tibetan_relations_during_the_Ming_dynasty,"Quando os mongóis Dzungar tentaram espalhar seu território do que hoje é Xinjiang para o Tibete, o Imperador Kangxi (r. 1661–1722) respondeu aos pedidos tibetanos de ajuda com sua própria expedição ao Tibete, ocupando Lhasa em 1720. Por volta de 1751, durante o reinado do Imperador Qianlong (r. 1735–1796), um protetorado e uma guarnição permanente da dinastia Qing foi estabelecida no Tibete. A partir de 1751, Albert Kolb escreve que &quot;as alegações chinesas de suserania sobre o Tibete datam desta época&quot;.",Quando o Imperador Kangxi ocupou Lhasa?,"{'text': ['1720'], 'answer_start': [227]}"
1,56cf7c234df3c31400b0d832,Frédéric_Chopin,"Os dois se tornaram amigos, e por muitos anos viveram em estreita proximidade em Paris, Chopin em 38 Rue de la Chaussée-d&#39;Antin, e Liszt no Hôtel de France na Rue Lafitte, a poucos quarteirões de distância. Eles se apresentaram juntos em sete ocasiões entre 1833 e 1841. O primeiro, em 2 de abril de 1833, foi em um concerto beneficente organizado por Hector Berlioz para sua falecida esposa de atriz shakespeariana Harriet Smithson, durante a qual eles tocaram Sonata em Fá Menor de George Onslow para dueto de piano . Mais tarde, as aparições conjuntas incluíram um concerto beneficente para a Benevolent Association of Polish Ladies, em Paris. Sua última aparição juntos em público foi para um concerto de caridade realizado para o Memorial Beethoven em Bonn, realizado no Salle Pleyel e no Conservatório de Paris em 25 e 26 de abril de 1841.",Quando foi a primeira vez que Liszt e Chopin se apresentaram juntos?,"{'text': ['2 de abril de 1833'], 'answer_start': [290]}"
2,56d5372f2593cc1400307ae3,2008_Sichuan_earthquake,"O terremoto deixou pelo menos 5 milhões de pessoas sem moradia, embora o número possa chegar a 11 milhões. Milhões de animais e uma quantidade significativa de agricultura também foram destruídos, incluindo 12,5 milhões de animais, principalmente aves. Na província de Sichuan, um milhão de porcos morreram em um total de 60 milhões. A empresa de modelagem de catástrofes AIR Worldwide informou estimativas oficiais das perdas das seguradoras em US $ 1 bilhão do terremoto; os danos totais estimados excedem US $ 20 bilhões. Ela valoriza Chengdu, na época com uma população urbana de 4,5 milhões de pessoas, em torno de US $ 115 bilhões, com apenas uma pequena parcela coberta por seguros.","Por causa do terremoto, quantas pessoas não tinham moradia?","{'text': ['pelo menos 5 milhões'], 'answer_start': [19]}"
3,56d347fb59d6e414001462a0,Frédéric_Chopin,"Em 3 de dezembro, Chopin reclamou da sua má saúde e da incompetência dos médicos em Maiorca: &quot;Três médicos me visitaram ... O primeiro disse que eu estava morto; o segundo disse que eu estava morrendo; e o terceiro disse que eu estava prestes a morrer&quot;. morrer.&quot; Ele também teve problemas em mandar seu piano de Pleyel para ele. Finalmente chegou de Paris em dezembro. Chopin escreveu para Pleyel em janeiro de 1839: &quot;Estou lhe enviando meus Prelúdios [Op. 28]. Acabei-os em seu pequeno piano, que chegou na melhor condição possível apesar do mar, do mau tempo e da Palma. costumes.&quot; Chopin também foi capaz de realizar trabalhos em sua Ballade No. 2, op. 38; dois Polonaises, op. 40; e o Scherzo No. 3, op. 39",Para quem Chopin enviou seus Prelúdios?,"{'text': ['Pleyel'], 'answer_start': [327]}"
4,5732a8641d5d2e14009ff886,Financial_crisis_of_2007%E2%80%9308,"Essas instituições, assim como certos bancos regulamentados, também assumiram encargos significativos de dívidas, ao mesmo tempo em que forneciam os empréstimos descritos acima e não tinham um colchão financeiro suficiente para absorver grandes inadimplências de empréstimos ou perdas de MBS. Essas perdas afetaram a capacidade das instituições financeiras de emprestar, desacelerando a atividade econômica. As preocupações em relação à estabilidade das principais insti

## Lets preprocess the data

tokenize the data

In [10]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint)

In [11]:
# this will make sure the tokenizer is fast
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
pad_on_right = tokenizer.padding_side == "right" # model except padding on the left

max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
    return tokenized_examples

## we need to prepare the data for multiple reason

*   if we have impossible answers like if they are in another feature with long context
*   we can solve that by set the cls index for start and end position or we can just discard those example if we set allow_impossible_answer = False




In [13]:
train_tokenize_data = train_data.map(prepare_train_features, batched=True, remove_columns=train_data.column_names)
validate_tokenize_data = validate_data.map(prepare_train_features, batched=True, remove_columns=validate_data.column_names)

Map:   0%|          | 0/10512 [00:00<?, ? examples/s]

Map:   0%|          | 0/5285 [00:00<?, ? examples/s]

In [14]:
train_tokenize_data

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 12942
})

## fine tune the model

In [15]:
model = RobertaForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

## Initiate training
### before that we need training arguments which will need folder name and other optional

In [16]:
args = TrainingArguments(
    f"test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

## data collector that will batch the examples together

In [17]:
data_collector = default_data_collator

## Trainer

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenize_data,
    eval_dataset=validate_tokenize_data,
    data_collator=data_collector,
    tokenizer=tokenizer,
)

In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.647500,2.053657
2,1.744800,2.026809
3,1.547300,2.010895


TrainOutput(global_step=2427, training_loss=1.879981469656147, metrics={'train_runtime': 3415.4929, 'train_samples_per_second': 11.368, 'train_steps_per_second': 0.711, 'total_flos': 7608830907773952.0, 'train_loss': 1.879981469656147, 'epoch': 3.0})

## Evaluation will need more work

### model predicts logits by itself for start and end position of our ans. so if we take a batch from validation dataloader, output will have start and end logit.

### we will use best possible answers limit with hyper parameter.
### wheterver we find within that limit, we will get the score and find the best one

In [20]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [21]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [22]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 85,  85,  15,  15,  25,  14, 114,  12,  14,  10, 114,  13, 143, 145,
          29,  20], device='cuda:0'),
 tensor([ 64,  87,  17,  17, 190,  16, 120,  14,  16,  12, 120,  15, 144, 146,
          31,  53], device='cuda:0'))

## now we will pre-process our validation a bit diferently than train
### because we need to check how can we check whether the given span is within the context and not the question.
### we need to do two task, 
*   add the id of the generated feature
*   add offset mapping which will give a map from token indices to character position



In [24]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [25]:
validation_features = validate_data.map(
    prepare_validation_features,
    batched=True,
    remove_columns=validate_data.column_names
)

Map:   0%|          | 0/5285 [00:00<?, ? examples/s]

## lets predict for all the validation features

In [26]:
raw_pred = trainer.predict(validation_features)

In [27]:
## seting the columns that are hidden since trainer hiddes unused columns
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

## double check the ans

In [31]:
validate_data[0]['answers']

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [182, 182, 182]}

## we will need a map between examples and their corresponding features. Also, since one example can give several features, 
## we will need to gather together all the answers in all the features generated by a given example, then pick the best one. 

In [32]:
examples = validate_data
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

## we can now predict the impossible ans when its score is greater than the score of the bes non-impossible ans

## post-processing

In [33]:



def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None 
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

## get the final prediction 

In [34]:
final_pred = postprocess_qa_predictions(validate_data, validation_features, raw_pred.predictions)

Post-processing 5285 example predictions split into 6702 features.


  0%|          | 0/5285 [00:00<?, ?it/s]

In [35]:
metric = load_metric("squad")

In [36]:
formatted_pred = [{"id": k, "prediction_text": v} for k, v in final_pred.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in validate_data]
metric.compute(predictions=formatted_pred, references=references)

{'exact_match': 34.361400189214756, 'f1': 45.94110320611071}